In [6]:
from ldap3 import Server, Connection, Tls
import ssl
from logger import logger

In [76]:
class LDAPConfig(object):
    def __init__(self, config=None):
        self.server_uri = None
        self.bind_dn = None
        self.password = None
        self.use_ssl = None
        self.search_ougroup = None
        self.search_filter = None
        self.attr_map = None
        self.load_from_settings()
    def load_from_test(self):
        self.server_uri = "labldap02.cloudapp.net"
        self.bind_dn = 'cn=testLAB,o=resources'
        self.password = 'Rc1234pfop'
        self.use_ssl = False
        self.search_ougroup = "ou=fixtures,o=test"
        self.search_filter = "(cn=%(user)s)"
        self.attr_map = {"username": "cn", "name": "sn", "email": "mail"}
        
    def load_from_settings(self):
        self.server_uri = "ldap.google.com"
        self.bind_dn = "GrayVicero"
        self.password = "SgSQZgbgLeY6XeY6u33XeTZ9"
        self.use_ssl = False
        self.search_ougroup = "ou=Employees,ou=Users,dc=mixwan,dc=net|ou=Managers,ou=Users,dc=mixwan,dc=net"
        self.search_filter = "(cn=%(user)s)"
        self.attr_map = {"username": "uid", "name": "sn", "email": "mail"}

        
class LDAPServerUtil(object):
    def __init__(self, config=None):
        self._conn = None
        self.search_users = None
        self.search_value = None
        self._paged_size = 1000
        self.config = LDAPConfig()
        
    @property
    def connection(self):
        if self._conn:
            return self._conn
        tls = Tls(local_private_key_file = 'data/certs/Google_2022_11_18_40607.key',
                  local_certificate_file = 'data/certs/Google_2022_11_18_40607.crt',
                 version = ssl.PROTOCOL_TLSv1)
        server = Server(self.config.server_uri, use_ssl=self.config.use_ssl, tls=tls)
        conn = Connection(server, self.config.bind_dn, self.config.password,
                        sasl_mechanism = 'EXTERNAL', sasl_credentials = 'username')
        conn.bind()
        self._conn = conn
        return self._conn
    
    
    def paged_cookie(self):
        if self._paged_size is None:
            return None
        cookie = self.connection.result['controls']['1.2.840.113556.1.4.319']['value']['cookie']
        return cookie
    
    def get_search_filter_extra(self):
        extra = ''
        if self.search_users:
            mapping_username = self.config.attr_map.get('username')
            for user in self.search_users:
                extra += '({}={})'.format(mapping_username, user)
            return '(|{})'.format(extra)
        if self.search_value:
            for attr in self.config.attr_map.values():
                extra += '({}={})'.format(attr, self.search_value)
            return '(|{})'.format(extra)
        return extra
    
    def get_search_filter(self):
        search_filter = self.config.search_filter % {'user': '*'}
        search_filter_extra = self.get_search_filter_extra()
        if search_filter_extra:
            search_filter = '(&{}{})'.format(search_filter, search_filter_extra)
        return search_filter

    def search_user_entries_ou(self, search_ou, paged_cookie=None):
        search_filter = self.get_search_filter()
        attributes = list(self.config.attr_map.values())
        ok = self.connection.search(
            search_base=search_ou, search_filter=search_filter,
            attributes=attributes, paged_size=self._paged_size,
#             paged_cookie=paged_cookie
        )
        if not ok:
            error = "Search no entry matched in ou {}".format(search_ou)
            logger.error(error)

    def search_user_entries(self):
        logger.info("Search user entries")
        user_entries = list()
        search_ous = str(self.config.search_ougroup).split('|')
        for search_ou in search_ous:
            logger.info("Search user entries ou: {}".format(search_ou))
            self.search_user_entries_ou(search_ou)
            user_entries.extend(self.connection.entries)
#             while self.paged_cookie():
#                 self.search_user_entries_ou(search_ou, self.paged_cookie())
#                 user_entries.extend(self.connection.entries)
        return user_entries

    def user_entry_to_dict(self, entry):
        user = {}
        attr_map = self.config.attr_map.items()
        for attr, mapping in attr_map:
            if not hasattr(entry, mapping):
                continue
            value = getattr(entry, mapping).value or ''
            if attr == 'is_active' and mapping.lower() == 'useraccountcontrol' \
                    and value:
                value = int(value) & LDAP_AD_ACCOUNT_DISABLE != LDAP_AD_ACCOUNT_DISABLE
            user[attr] = value
#         user['username'] = user['username'][0]
#         print("user: ",user)
        return user

    def user_entries_to_dict(self, user_entries):
        users = []
        for user_entry in user_entries:
            user = self.user_entry_to_dict(user_entry)
            users.append(user)
        return users

    def search(self, search_users=None, search_value=None):
        logger.info("Search ldap users")
        self.search_users = search_users
        self.search_value = search_value
        user_entries = self.search_user_entries()
        users = self.user_entries_to_dict(user_entries)
        return users


In [77]:
userserver = LDAPServerUtil()
#userserver.paged_cookie()
userserver.connection.result

{'result': 13,
 'description': 'confidentialityRequired',
 'dn': '',
 'message': '',
 'referrals': None,
 'saslCreds': None,
 'type': 'bindResponse'}

In [75]:
userserver.search()

[21/Nov/2019 19:44:44] INFO - Search ldap users
[21/Nov/2019 19:44:44] INFO - Search user entries
[21/Nov/2019 19:44:44] INFO - Search user entries ou: ou=Employees,ou=Users,dc=mixwan,dc=net
[21/Nov/2019 19:44:44] INFO - Search user entries ou: ou=Managers,ou=Users,dc=mixwan,dc=net


[{'username': 'li.bowen', 'name': 'Li', 'email': 'li.bowen@mixwan.net'},
 {'username': 'lu.junpeng', 'name': 'Lu', 'email': 'lu.junpeng@mixwan.net'},
 {'username': 'h', 'name': 'Wong', 'email': 'h@mixwan.net'},
 {'username': 'luye', 'name': 'Lu', 'email': 'luye@mixwan.net'},
 {'username': 'w', 'name': 'W', 'email': 'w@mixwan.net'}]

In [62]:
userserver.user_entry_to_dict(userserver.connection.entries)

{}

In [63]:
userserver.connection.entries

[DN: uid=w,ou=Managers,ou=Users,dc=mixwan,dc=net - STATUS: Read - READ TIME: 2019-11-21T19:37:18.574489
     cn: w
         QF W
     mail: w@mixwan.net
     sn: W,
 DN: uid=luye,ou=Managers,ou=Users,dc=mixwan,dc=net - STATUS: Read - READ TIME: 2019-11-21T19:37:18.574623
     cn: luye
         Ye Lu
     mail: luye@mixwan.net
     sn: Lu,
 DN: uid=h,ou=Managers,ou=Users,dc=mixwan,dc=net - STATUS: Read - READ TIME: 2019-11-21T19:37:18.574746
     cn: h
         H. W. Wong
     mail: h@mixwan.net
     sn: Wong]

In [38]:
'(cn=%(user)s)' % {'user': '*'}

'(cn=*)'